In [20]:
import pandas as pd
from datetime import datetime

## Process Dataset

### Load Dataset
First we read the CSV file and preview its contents with `head()`.

In [21]:
df = pd.read_csv('datasets/us_accidents_original.csv')
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-2047758,Source2,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,...,False,False,False,False,True,False,Day,Day,Day,Day
1,A-4694324,Source1,2,2022-12-03 23:37:14.000000000,2022-12-04 01:56:53.000000000,38.990562,-77.399070,38.990037,-77.398282,0.056,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-5006183,Source1,2,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,...,False,False,False,False,True,False,Day,Day,Day,Day
3,A-4237356,Source1,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,...,False,False,False,False,False,False,Night,Night,Night,Night


Next we take a look at all the features it has and the initial number.

In [22]:
def check_features(data):
    print('Columns:', data.columns)
    print('Count:', len(data.columns))

check_features(df)

Columns: Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description',
       'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')
Count: 46


We're a long way from 150+ but we can use feature engineering to work with it.

### Pruning Unecessary Features
Although we're looking to get our feature numbers up, we need to first remove features that are completely irrelevant to our project. At a glance, `ID` and `Source` are purely metadata and don't contribute any information to the accidents themselves. Therefore they can be removed from our dataset.

In [23]:
irrelevant_features = [
    'ID',
    'Source'
]

df.drop(columns=irrelevant_features, inplace=True)

### One-Hot Encoding
There's a lot of categorical columns here that need to be encoded as numerical
so our ML models can train on our data more easily and accurately. The main way
we will do this is with one-hot encoding, which splits the categorical columns
into a binary column for each possible value.

At a glance, some columns that should be one-hot encoded are `State`, `Timezone`, and `Weather_Condition`.

In [24]:
df = pd.get_dummies(df, columns=['State', 'Weather_Condition'])
check_features(df)

Columns: Index(['Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Street',
       ...
       'Weather_Condition_Thunder and Hail',
       'Weather_Condition_Thunder in the Vicinity',
       'Weather_Condition_Thunderstorm',
       'Weather_Condition_Thunderstorms and Rain', 'Weather_Condition_Tornado',
       'Weather_Condition_Volcanic Ash', 'Weather_Condition_Widespread Dust',
       'Weather_Condition_Widespread Dust / Windy',
       'Weather_Condition_Wintry Mix', 'Weather_Condition_Wintry Mix / Windy'],
      dtype='object', length=199)
Count: 199


### Encoding Datetime
Currently, we have start and end times encoded as `mm/dd/yyyy hh:mm:ss`. However, some timestamps have milliseconds while others do not (ex: `2022-12-03 23:37:14.000000000`). Since most of these millisecond values are just 0's, we'll just remove millisecond measures and restrict timestamp precision to just seconds.


In [25]:
df.rename(columns={'Start_Time': 'Start_Datetime', 'End_Time': 'End_Datetime'}, inplace=True)

def remove_milliseconds(datetime_str):
    if '.' in datetime_str:
        datetime_str = datetime_str.split('.')[0]
    return datetime_str

df['Start_Datetime'] = [remove_milliseconds(time) for time in df['Start_Datetime']]
df['End_Datetime'] = [remove_milliseconds(time) for time in df['End_Datetime']]

df.head()

,Severity,Start_Datetime,End_Datetime,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,...,Weather_Condition_Thunder and Hail,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Tornado,Weather_Condition_Volcanic Ash,Weather_Condition_Widespread Dust,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_Wintry Mix / Windy
0,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,Accident on LA-19 Baker-Zachary Hwy at Lower Z...,Highway 19,...,False,False,False,False,False,False,False,False,False,False
1,2,2022-12-03 23:37:14,2022-12-04 01:56:53,38.990562,-77.399070,38.990037,-77.398282,0.056,Incident on FOREST RIDGE DR near PEPPERIDGE PL...,Forest Ridge Dr,...,False,False,False,False,False,False,False,False,False,False
2,2,2022-08-20 13:13:00,2022-08-20 15:22:45,34.661189,-120.492822,34.661189,-120.492442,0.022,Accident on W Central Ave from Floradale Ave t...,Floradale Ave,...,False,False,False,False,False,False,False,False,False,False
3,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,Incident on I-90 EB near REST AREA Drive with ...,14th St NW,...,False,False,False,False,False,False,False,False,True,False
4,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,RP ADV THEY LOCATED SUSP VEH OF 20002 - 726 CR...,River Blvd,...,False,False,False,False,False,False,False,False,False,False


Next, we will split off date and time into their own features to make it easier for analysis, especially since we care a lot more about time than date.

In [26]:
def get_date(datetime_str):
    return datetime_str.split()[0]

def get_time(datetime_str):
    return datetime_str.split()[1]

df['Start_Date'] = [get_date(datetime_str) for datetime_str in df['Start_Datetime']]
df['Start_Time'] = [get_time(datetime_str) for datetime_str in df['Start_Datetime']]
df['End_Date'] = [get_date(datetime_str) for datetime_str in df['End_Datetime']]
df['End_Time'] = [get_time(datetime_str) for datetime_str in df['End_Datetime']]

Finally, we can calculate the elapsed time and encode it in a new feature as well.

In [27]:
def calculate_elapsed_time(start_times, end_times, time_format='%Y-%m-%d %H:%M:%S'):
    elapsed_times = []
    for start, end in zip(start_times, start_times):
        start_dt = datetime.strptime(start, time_format)
        end_dt = datetime.strptime(end, time_format)
        elapsed_times.append(abs(end_dt - start_dt))
    return pd.Series(elapsed_times)

df['Elapsesd_Time'] = calculate_elapsed_time(df['Start_Datetime'], df['End_Datetime'])

### Weekday vs Weekend

Another feature we can encode is if the traffic accident ocurred on a weekday or weekend. We will check only the start date since we are most concerned with the moment the accident occurred.

In [28]:
df['Is_Weekend'] = pd.to_datetime(df['Start_Date']).dt.weekday >= 5
df.head()

,Severity,Start_Datetime,End_Datetime,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,...,Weather_Condition_Widespread Dust,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_Wintry Mix / Windy,Start_Date,Start_Time,End_Date,End_Time,Elapsesd_Time,Is_Weekend
0,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,Accident on LA-19 Baker-Zachary Hwy at Lower Z...,Highway 19,...,False,False,False,False,2019-06-12,10:10:56,2019-06-12,10:55:58,0 days,False
1,2,2022-12-03 23:37:14,2022-12-04 01:56:53,38.990562,-77.399070,38.990037,-77.398282,0.056,Incident on FOREST RIDGE DR near PEPPERIDGE PL...,Forest Ridge Dr,...,False,False,False,False,2022-12-03,23:37:14,2022-12-04,01:56:53,0 days,True
2,2,2022-08-20 13:13:00,2022-08-20 15:22:45,34.661189,-120.492822,34.661189,-120.492442,0.022,Accident on W Central Ave from Floradale Ave t...,Floradale Ave,...,False,False,False,False,2022-08-20,13:13:00,2022-08-20,15:22:45,0 days,True
3,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,Incident on I-90 EB near REST AREA Drive with ...,14th St NW,...,False,False,True,False,2022-02-21,17:43:04,2022-02-21,19:43:23,0 days,False
4,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,RP ADV THEY LOCATED SUSP VEH OF 20002 - 726 CR...,River Blvd,...,False,False,False,False,2020-12-04,01:46:00,2020-12-04,04:13:09,0 days,False


### Final Feature Count
Taking a look at our final feature count, we have:

In [29]:
check_features(df)

Columns: Index(['Severity', 'Start_Datetime', 'End_Datetime', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Street',
       ...
       'Weather_Condition_Widespread Dust',
       'Weather_Condition_Widespread Dust / Windy',
       'Weather_Condition_Wintry Mix', 'Weather_Condition_Wintry Mix / Windy',
       'Start_Date', 'Start_Time', 'End_Date', 'End_Time', 'Elapsesd_Time',
       'Is_Weekend'],
      dtype='object', length=205)
Count: 205


### Missing Values
Before we can start our data analysis, we need to first check for missing values.
The goal here is to impute missing data where we can. We won't be dropping any records
with missing values since we want to handle it on a case-by-case decision for each
analysis.

First, we need to view which features contain missing values.

In [30]:
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]
missing_counts

End_Lat                  220377
End_Lng                  220377
Description                   1
Street                      691
City                         19
Zipcode                     116
Timezone                    507
Airport_Code               1446
Weather_Timestamp          7674
Temperature(F)            10466
Wind_Chill(F)            129017
Humidity(%)               11130
Pressure(in)               8928
Visibility(mi)            11291
Wind_Direction            11197
Wind_Speed(mph)           36987
Precipitation(in)        142616
Sunrise_Sunset             1483
Civil_Twilight             1483
Nautical_Twilight          1483
Astronomical_Twilight      1483
dtype: int64

We have a lot of missing values for the accident end coordinates (latitude and longitude).
If the corresponding distance is 0, we can just impute the end coordinates with the start.

In [31]:
zero_distance = df['Distance(mi)'] == 0
df.loc[zero_distance, ['End_Lat', 'End_Lng']] = df.loc[zero_distance, ['Start_Lat', 'Start_Lng']].values

Next, we can look at the `Weather_Timestamp`. Most records have the weather timestamp
and accident start timestamp be really close, usually within a few minutes of each other.
Therefore, we can just impute missing weather timestamps with the accident's start timestamp.

In [32]:
df['Weather_Timestamp'] = df['Weather_Timestamp'].fillna(df['Start_Datetime'])

Next up is the temperature and wind chill. After looking at the data, these 2 values
are typically very similar, being equal in many cases. Therefore, we can just impute
them with each other.

In [33]:
df['Temperature(F)'] = df['Temperature(F)'].fillna(df['Wind_Chill(F)'])
df['Wind_Chill(F)'] = df['Wind_Chill(F)'].fillna(df['Temperature(F)'])

Finally, we can look at the day/night indicators (`Sunrise_Sunset`, `Civil_Twilight`,
`Nautical_Twilight`, `Astronomical_Twilight`). We were originally going to impute
them with the majority of the other 3, but it turns out that every record either
values for all of them, or none of them.

In [34]:
daylight = ['Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
df[df[daylight].isna().any(axis=1) & ~df[daylight].isna().all(axis=1)][daylight] 


,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight


### Export to CSV
*Note: remove this step in the final notebook since it's unnecessary*

Our final dataset looks something like this:

In [35]:
df.head()

,Severity,Start_Datetime,End_Datetime,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,...,Weather_Condition_Widespread Dust,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_Wintry Mix / Windy,Start_Date,Start_Time,End_Date,End_Time,Elapsesd_Time,Is_Weekend
0,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,30.641211,-91.153481,0.000,Accident on LA-19 Baker-Zachary Hwy at Lower Z...,Highway 19,...,False,False,False,False,2019-06-12,10:10:56,2019-06-12,10:55:58,0 days,False
1,2,2022-12-03 23:37:14,2022-12-04 01:56:53,38.990562,-77.399070,38.990037,-77.398282,0.056,Incident on FOREST RIDGE DR near PEPPERIDGE PL...,Forest Ridge Dr,...,False,False,False,False,2022-12-03,23:37:14,2022-12-04,01:56:53,0 days,True
2,2,2022-08-20 13:13:00,2022-08-20 15:22:45,34.661189,-120.492822,34.661189,-120.492442,0.022,Accident on W Central Ave from Floradale Ave t...,Floradale Ave,...,False,False,False,False,2022-08-20,13:13:00,2022-08-20,15:22:45,0 days,True
3,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,Incident on I-90 EB near REST AREA Drive with ...,14th St NW,...,False,False,True,False,2022-02-21,17:43:04,2022-02-21,19:43:23,0 days,False
4,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,RP ADV THEY LOCATED SUSP VEH OF 20002 - 726 CR...,River Blvd,...,False,False,False,False,2020-12-04,01:46:00,2020-12-04,04:13:09,0 days,False


In [36]:
df.to_csv('datasets/us_accidents_expanded.csv', index=False)

I'll also just be making a tiny version of the dataset for feature navigation purposes.

In [37]:
df_small = df.head(100)
df_small.to_csv('datasets/us_accidents_small.csv', index=False)